<a href="https://colab.research.google.com/github/therobinkay/firstmover/blob/main/First_mover_advantage_SI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!cd "drive/My Drive"

In [ ]:
# import necessary packages

import io
import pandas as pd
import os
import itertools
import numpy as np
import scipy.stats
import time

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

import math

from collections import defaultdict
from itertools import combinations as comb

import networkx as nx
import random

In [ ]:
# read necessary data files

data = pd.read_csv("drive/My Drive/codes/data.csv")
dp = pd.read_csv("drive/My Drive/codes/doipacs.csv")
cen = pd.read_csv('drive/My Drive/codes/cen.csv')
cdata = pd.read_csv('drive/My Drive/codes/cdata.csv')
cb = pd.read_csv("drive/My Drive/codes/citationBara.csv")

In [ ]:
############## SELECT DESIRED PACS (HOMOPHILY ONLY) ###############

# Select subfield (0-9)
N = 0

homn = pd.read_csv(f"drive/My Drive/codes/hom{N}.csv")

# q thresholds by subfield
qs = [0.002, 0.003, 0.003, 0.002, 0.0025, 0.006, 0.0018, 0.002, 0.0025, 0.006]

# print(len(homn))
# homn.head()

# Supplementary Materials

## 1. Productivity Plots

In [ ]:
# Creating a DataFrame with productivity calculated
prod = data.groupby(['id', 'gender']).agg({'year': [min, max, 'count']})

prod['career_age'] = prod['year']['max'] - prod['year']['min']
prod['prod'] = prod['year']['count'] / (prod['career_age'])
prod = prod[prod['career_age'] != 0]

prod.sort_values([('year','count')], ascending=False).head()

# Separating the DataFrame by gender
prod_m = prod.xs('male', level='gender').sort_values(
    by=['prod'],ascending=False, ignore_index=True)
prod_fm = prod.xs('female', level='gender').sort_values(
    by=['prod'],ascending=False)

# Career Age by gender
camale = prod_m.groupby(['career_age'])['career_age'].count()
cafemale = prod_fm.groupby(['career_age'])['career_age'].count()

In [ ]:
# Career age vs. Number of authors, separated by gender
fig, ax = plt.subplots(figsize=(6,4.5))
right_side = ax.spines["right"]
top_side = ax.spines["top"]
right_side.set_visible(False)
top_side.set_visible(False)

# Male authors
camale.plot(color='#67a9cf', lw=3)
# Female authors
cafemale.plot(color='#ef8a62', lw=3)

plt.xticks(fontsize=20)
plt.xlim(1,35)
plt.yticks(fontsize=20)
plt.legend(['male', 'female'], loc='upper right', fontsize=20)
plt.xlabel('Career Age', fontsize=20)
plt.ylabel('Number of authors', fontsize=20)
plt.tight_layout()

In [ ]:
# creating CDF data for productivity

from scipy.interpolate import make_interp_spline, BSpline
from scipy.interpolate import interp1d

cdf1=np.arange(len(prod_m))/float(len(prod_m)-1)
cdf2=np.arange(len(prod_fm))/float(len(prod_fm)-1)

In [ ]:
# Productivity CDF Plots
fig, ax = plt.subplots(figsize=(6,4.5))
right_side = ax.spines["right"]
top_side = ax.spines["top"]
right_side.set_visible(False)
top_side.set_visible(False)

# Male authors
plt.plot(prod_m['prod'],1-cdf1, color='#67a9cf', lw=2)
# Female authors
plt.plot(prod_fm['prod'],1-cdf2, color='#ef8a62', lw=2)

plt.xlabel('Productivity', fontsize=20)
plt.xticks(fontsize=20)
plt.xlim(xmin=0, xmax=4)
plt.ylabel('Probability', fontsize=20)
plt.yticks(fontsize=20)
plt.legend(['male', 'female'], loc='lower right', fontsize=20)
plt.tight_layout()

# Use the following xlim, ylim instead for zoomed-in plot
# plt.xlim(xmin=0.45, xmax=0.55)
# plt.ylim(ymin=0.15, ymax=0.25)

## 2. Statistical Tests comparing degree centrality among top ranks

In [ ]:
# non-alphabetically ordered primary gender paper
na = data[data['is_alpha'] == False]
na_p = na[na['order'] == 1]

# consider degree centrality > 0
vc = cb['cited_doi'].value_counts()
cb_sub = vc[vc.to_numpy() > 0].to_frame()

cited = pd.merge(cb_sub, na_p, left_index = True, right_on = 'doi')

# observe male and female nodes
cited_m = cited[cited['gender'] == 'male']
cited_fm = cited[cited['gender'] == 'female']

# unique number of degree centrality
cu = cited['cited_doi'].unique()

# Insert desired ranks for xxx:

# print(len(cited_m[cited_m['cited_doi'] >= cu[len(cu)*xxx]]))
# print(len(cited_fm[cited_fm['cited_doi'] >= cu[len(cu)*xxx]]))

## 3. Difference in received citations among similar pairs

In [ ]:
# apply nx.graph to citation relationship network
Graphtype = nx.DiGraph()

C = nx.from_pandas_edgelist(cen, source='citing_doi',
                            target='cited_doi', create_using=Graphtype)

# observe author id & gender (primary author)
nodedata = data.query('is_alpha == False & order == 1').drop(
    ['order', 'numAuthor',	'is_last',	'is_alpha',	'year',
    'articleType',	'journal'], axis=1)

# assign PageRank centrality to each paper
pr = nx.pagerank(C)

# create a centrality dataframe
prs = pd.DataFrame()
prs['doi'] = pr.keys()
prs['PRcen'] = pr.values()

In [ ]:
# Calculate (male centrality - female centrality)

# Create a dataframe concerning both centrality info of 2 papers
prs_sim = prs.merge(nodedata, left_on='doi', right_on='Label',
                    how='left').drop(['Label'], axis=1)
prs_sim = prs_sim[["doi", 'id', 'Gender', 'centrality']].sort_values(
    'centrality', ascending=False)

ph = pd.merge(homn, prs_sim, left_on = "paper1", right_on = "doi",
              how = "left").drop(["doi", "id", "Gender"], axis=1).rename(
                  columns={"centrality": "cen1"})

ph = ph.merge(prs_sim, left_on = "paper2", right_on = "doi",
              how = "left").drop(["doi", "id", "Gender"], axis=1).rename(
                  columns={"centrality": "cen2"})

# consider only m-f pairs
ph = ph[ph['gender1'] != ph['gender2']].reset_index(drop=True)

# get the difference column
listd = []

for c in range(len(ph)):
  if ph['gender1'][c] == 'male':
    listd.append(ph['cen1'][c] - ph['cen2'][c])
  else:
    listd.append(ph['cen2'][c] - ph['cen1'][c])
ph['cen_d'] = listd

In [ ]:
# Extract subDataFrame with q-value less than p threshold

p = qs[N]

phsub = ph[ph['qval'] < p].sort_values('cen_d').reset_index(drop=True)

# calculate for both male and female
mlist = []
fmlist = []

for i in range(len(phsub)):
  if phsub['gender1'][i] == 'male':
    mlist.append(phsub['cen1'][i])
    fmlist.append(phsub['cen2'][i])
  else:
    mlist.append(phsub['cen2'][i])
    fmlist.append(phsub['cen1'][i])

# Find the standard deviation
sd = math.sqrt((np.std(mlist) ** 2 + np.std(fmlist) ** 2)/len(phsub))

# Find z-scores (for p-values)
print((np.mean(mlist)-np.mean(fmlist))/sd)

## 4. Number of sampled similar pairs by year

In [ ]:
## Load data
y_lst = []
n_lst = []
with open("../data/se_count.csv","r") as f:
    print (next(f))
    for l in f.readlines():
        l_lst = l.split(",")
        y_lst.append(int(l_lst[0]))
        n_lst.append(int(l_lst[1]))

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import mark_inset, inset_axes

## Create small figure so typography is scaled up
plt.figure(figsize=(.5*8,.5*6))

ax = plt.axes()
plt.plot(y_lst[3:], n_lst[3:], "-o",color="k")

plt.xlim(1985,2010)
plt.xlabel("Year")
plt.ylabel("Count")

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

## Create inset axis
axins = inset_axes(ax, 
                   width="70%",
                   height="70%", 
                   bbox_to_anchor=(-.00, .3, .6, .7),
                   bbox_transform=ax.transAxes)
plt.sca(axins) ## sca=set current axes

plt.plot(y_lst[3:9], n_lst[3:9], "-o",color="k")

## ylim and xlim will control both the span of the inset axes 
## and the rectangle drawn in main axes
plt.ylim(0,350)
plt.xlim(1985.5,1991.5)

plt.xticks([1986,1988,1990],rotation=45) ## note the rotation
plt.locator_params(axis='y', nbins=4)

## Tool to mark the zoomed part
mark_inset(ax, axins, 
           loc1=2, loc2=4, 
           fc="none", 
           ec="0.5",
           zorder=0,
          lw=1)

plt.tight_layout()
plt.show()

## 5. Percentile Plots of difference by Year

In [ ]:
# DataFrame: {citing_doi, citing_year, cited_doi}
citinfo = data[["doi", "year"]].drop_duplicates(subset = 'doi')
cols = ['citing_doi', 'citing_year', 'cited_doi']

citinfo = citinfo.merge(cb, left_on = 'doi', right_on = 'citing_doi',
              how = 'inner').drop(['doi'], axis=1).rename(
                  {'year': 'citing_year'}, axis=1).reindex(
                      columns=cols).sort_values('cited_doi')

# DataFrame: info among different gender pair
sim_mf = pair[pair["gender1"] != pair["gender2"]].reset_index(drop=True)

# Create a dummy data (test) to perform SQL:
# Create a DataFrame that treats citation info for male and female papers
test = sim_mf.merge(citinfo, left_on = 'paper1',
                    right_on = 'cited_doi', how = 'left').drop(
                        ["cited_doi", "citing_year"], axis=1).drop_duplicates(
                            subset=['paper1', 'citing_doi'])
test['count'] = test.groupby(['paper1'])['paper1'].transform('count')
test = test.drop(["citing_doi"], axis=1).rename({'count': 'count1'}, axis=1)

test = test.drop_duplicates(subset=['paper1', 'paper2']).drop(
    ['gender1', 'year1', 'paper2', 'gender2', 'year2', 'qval', 'k'], axis=1)
sim_mf = sim_mf.merge(test, on='paper1', how='inner')

test = sim_mf.merge(citinfo, left_on = 'paper2',
                    right_on = 'cited_doi', how = 'left').drop(
                        ["cited_doi", "citing_year"], axis=1).drop_duplicates(
                            subset=['paper2', 'citing_doi'])
test['count'] = test.groupby(['paper2'])['paper2'].transform('count')
test = test.drop(["citing_doi"], axis=1).rename(
    {'count': 'count2'}, axis=1).drop_duplicates(
        subset=['paper1', 'paper2']).drop(
            ['gender1', 'year1', 'paper1', 'gender2', 'count1',
             'year2', 'qval', 'k'], axis=1)
sim_mf = sim_mf.merge(test, on='paper2', how='inner')

# Delete the dummy data
del test

In [ ]:
# Assign latter year as a publication year

listc = []

for j in range(len(sim_mf)):
  listc.append(max(sim_mf['year1'][j], sim_mf['year2'][j]))

sim_mf['year'] = listc

# Calculate year & centrality difference among pairs

listd = []
listy = []

for c in range(len(sim_mf)):
  if sim_mf['gender1'][c] == 'male':
    listd.append(sim_mf['count1'][c] - sim_mf['count2'][c])
    listy.append(sim_mf['year1'][c] - sim_mf['year2'][c])
  else:
    listd.append(sim_mf['count2'][c] - sim_mf['count1'][c])
    listy.append(sim_mf['year2'][c] - sim_mf['year1'][c])

sim_mf['countd'] = listd
sim_mf['yeard'] = listy

# Restrict the year difference to 3 years
sim_mf = sim_mf[sim_mf['yeard'] > -4].sort_values(
    'yeard').reset_index(drop=True)
sim_mf = sim_mf[sim_mf['yeard'] < 4].sort_values(
    'yeard').reset_index(drop=True)

# Set the set p-threshold
p = 0.002

# Treat pairs with qval less than p-threshold only
sim_mf = sim_mf[sim_mf['qval'] < p].sort_values('countd').reset_index(drop=True)

# Extract a subdataframe
sim_mf = sim_mf[['year', 'countd']].sort_values(by=['year', 'countd'])

In [ ]:
# Apply the identical process for M-M pairs

# Assign latter year as a publication year
listc = []

for j in range(len(sim_mm)):
  listc.append(max(sim_mm['year1'][j], sim_mm['year2'][j]))

sim_mm['year'] = listc

# Calculate year & centrality difference among pairs
listd = []
listy = []

for c in range(len(sim_mm)):
  if sim_mm['gender1'][c] == 'male':
    listd.append(sim_mm['count1'][c] - sim_mm['count2'][c])
    listy.append(sim_mm['year1'][c] - sim_mm['year2'][c])
  else:
    listd.append(sim_mm['count2'][c] - sim_mm['count1'][c])
    listy.append(sim_mm['year2'][c] - sim_mm['year1'][c])

sim_mm['countd'] = listd
sim_mm['yeard'] = listy

# Restrict the year difference to 3 years
sim_mm = sim_mm[sim_mm['yeard'] > -4].sort_values(
    'yeard').reset_index(drop=True)
sim_mm = sim_mm[sim_mm['yeard'] < 4].sort_values(
    'yeard').reset_index(drop=True)

# Set the set p-threshold
p = 0.001

# Treat pairs with qval less than p-threshold only
sim_mm = sim_mm[sim_mm['qval'] < p].sort_values(
    'countd').reset_index(drop=True)

# Extract a subdataframe
sim_mm = sim_mm[['year', 'countd']].sort_values(by=['year', 'countd'])

In [ ]:
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

# Get percentiles for M-F pairs
mfpair = mf.groupby('year').agg([np.mean, percentile(10), percentile(20),
                                 percentile(30), percentile(40), np.median,
                                 percentile(60), percentile(70), percentile(80),
                                 percentile(90)]).reset_index()

In [ ]:
# Plot Publication Year vs. Citation Difference for M-F pairs

fig, ax = plt.subplots(figsize=(6,4.5))
right_side = ax.spines["right"]
top_side = ax.spines["top"]
right_side.set_visible(False)
top_side.set_visible(False)

# color code
cm = sns.color_palette("Reds",10)

# equilibrium line
plt.axhline(y=0, color='black', linestyle='dotted', lw=3)

# Plot the median
plt.plot(mfpair.year,mfpair.countd["median"],"-",label="M-F Median",
         color=cm[9], lw=3)

# Fill in the percentiles
plt.fill_between(mfpair.year, mfpair.countd["percentile_10"],
                 mfpair.countd["percentile_20"], color=cm[1])
plt.fill_between(mfpair.year, mfpair.countd["percentile_20"],
                 mfpair.countd["percentile_30"], color=cm[3])
plt.fill_between(mfpair.year, mfpair.countd["percentile_30"],
                 mfpair.countd["percentile_40"], color=cm[5])
plt.fill_between(mfpair.year, mfpair.countd["percentile_40"],
                 mfpair.countd["median"], color=cm[7])
plt.fill_between(mfpair.year, mfpair.countd["median"],
                 mfpair.countd["percentile_60"], color=cm[7])
plt.fill_between(mfpair.year, mfpair.countd["percentile_60"],
                 mfpair.countd["percentile_70"], color=cm[5])
plt.fill_between(mfpair.year, mfpair.countd["percentile_70"],
                 mfpair.countd["percentile_80"], color=cm[3])
plt.fill_between(mfpair.year, mfpair.countd["percentile_80"],
                 mfpair.countd["percentile_90"], color=cm[1])

plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel("Publication Year", fontsize=20)
plt.ylabel("\u0394(Citation)", fontsize=20)
plt.xlim(xmin=1986, xmax=2009)

plt.tight_layout()

In [ ]:
# Create a reference list for each year of M-F pair publication
ref = sim_mf.year.unique()
numlist = []

# mm is the DataFrame for the final plot (M-M pairs)
mm = pd.DataFrame(columns=['year'])

for num in range(1983,2010):
  numlist.append(num)
mm['year'] = numlist

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Getting a mean of means with 100 simulations for M-M pairs
for k in range(100):
  df = pd.DataFrame(columns=['year', 'countd'])
  vlist = []

  for i in range(len(ref)):
    appended_df = sim_mm[sim_mm['year'] == ref[i]].sample(
        sim_mf.year.value_counts().sort_index().iloc[i], replace=True)
    df = pd.concat([df,appended_df])
    
  for j in range(len(df)):
    if randint(0,1) == 0:
      vlist.append(df.countd.iloc[j])
    else:
      vlist.append(df.countd.iloc[j] * -1)
    
  df['value'] = vlist
  df = df.groupby("year").agg(
      [np.mean]).reset_index().drop(['year'], axis=1)

  mm = pd.concat([mm, df], axis=1)

In [ ]:
# Create data columns for all percentiles for M-M pairs

mmpair = mm.copy()

list_1 = []
list_2 = []
list_3 = []
list_4 = []
mlist = []
list_6 = []
list_7 = []
list_8 = []
list_9 = []

for i in range(len(mmpair)):
  list_1.append(np.percentile(mmpair.iloc[i,1:],10))
  list_2.append(np.percentile(mmpair.iloc[i,1:],20))
  list_3.append(np.percentile(mmpair.iloc[i,1:],30))
  list_4.append(np.percentile(mmpair.iloc[i,1:],40))
  mlist.append(np.median(mmpair.iloc[i,1:]))
  list_6.append(np.percentile(mmpair.iloc[i,1:],60))
  list_7.append(np.percentile(mmpair.iloc[i,1:],70))
  list_8.append(np.percentile(mmpair.iloc[i,1:],80))
  list_9.append(np.percentile(mmpair.iloc[i,1:],90))

mmpair['percentile_10'] = list_1
mmpair['percentile_20'] = list_2
mmpair['percentile_30'] = list_3
mmpair['percentile_40'] = list_4
mmpair['median'] = mlist
mmpair['percentile_60'] = list_6
mmpair['percentile_70'] = list_7
mmpair['percentile_80'] = list_8
mmpair['percentile_90'] = list_9

# Drop unnecessary columns
mmpair = mmpair[['year', 'percentile_10','percentile_20','percentile_30',
                 'percentile_40','median','percentile_60','percentile_70',
                 'percentile_80','percentile_90']]

In [ ]:
# Plot Publication Year vs. Citation Difference for M-M pairs

fig, ax = plt.subplots(figsize=(6,4.5))
right_side = ax.spines["right"]
top_side = ax.spines["top"]
right_side.set_visible(False)
top_side.set_visible(False)

# color code
cm2 = sns.color_palette("Blues",10)

# equilibrium line
plt.axhline(y=0, color='black', linestyle='dotted', lw=3)

# Plot the median
plt.plot(mmpair.year,mmpair["median"],"-",label="M-M Median",
         color=cm2[9], lw=3)

# Fill in the percentiles
plt.fill_between(mmpair.year, mmpair["percentile_10"],
                 mmpair["percentile_20"], color=cm2[1])
plt.fill_between(mmpair.year, mmpair["percentile_20"],
                 mmpair["percentile_30"], color=cm2[3])
plt.fill_between(mmpair.year, mmpair["percentile_30"],
                 mmpair["percentile_40"], color=cm2[5])
plt.fill_between(mmpair.year, mmpair["percentile_40"],
                 mmpair["median"], color=cm2[7])
plt.fill_between(mmpair.year, mmpair["median"],
                 mmpair["percentile_60"], color=cm2[7])
plt.fill_between(mmpair.year, mmpair["percentile_60"],
                 mmpair["percentile_70"], color=cm2[5])
plt.fill_between(mmpair.year, mmpair["percentile_70"],
                 mmpair["percentile_80"], color=cm2[3])
plt.fill_between(mmpair.year, mmpair["percentile_80"],
                 mmpair["percentile_90"], color=cm2[1])

plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel("Publication Year", fontsize=20)
plt.ylabel("\u0394(Citation)", fontsize=20)

plt.xlim(xmin=1983, xmax=2009)

plt.tight_layout()